# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 2)

## 1. Introduction

Being the son of migrant workers having to relocate was a constant through my childhood and even now when trying to find employment. For most of the moves we did growing up there seemed to be a relative or family friend living in our next destination. This provided me and my family a sense of camaraderie, a sense of belonging even as we travel and joined new communities. As such to make it easier to other Latinos to find a place of belonging this project aims to cluster and segment Kansas City's neighborhoods to determine potential locations to live or enjoy a night out.

Hispanic migration to the Kansas City area dates as far back as the 1830s, when the Santa Fe Trail opened up trade from Mexico. But the first major wave of immigration came with the establishment of railroads in Kansas City, Kan., and Kansas City, Mo. In the past 20 years, turmoil in Central America and Mexico has brought a new wave of immigration to the Kansas City area, revitalizing the historic Hispanic communities.

Target audiences for this project does not limit to Latino and Hispanic people but everyone. People could simply decide to look for a restaurant all the time because they are enjoy the food. 


## 2. Data

### Data Required to resolve the problem

In order to make a good choice in finding a location to reside in the Kansas City area the following data is required:
1. List/Information on neighborhoods from Kansas City with their respective Geodata (latitude and longitude)
2. Listed apartments for rent in Kansas City area
4. Venues and amenities in the Kansas City neighborhoods (e.g. top 10) 

This data will provide a set of parameters to determine the location of Latino centric communities where anyone can relocate. Determining the venues in these neighborhoods will allow others who only seek for a place to eat or enjoy a night out with information to make their decision.

## 3. Methodology

After acquiring a data set the CSV file was formated to include only the data of importance to determine the neighborhoods of interest. This was done initially by uploading the dataset, this was then read using the pandas library and converted onto a data-frame for further analysis and manipulation. This was followed by establishing a map with an overlay of all the neighborhoods in the list using the Geodata and Nominatim. To determine potential neighborhoods to live location data using the foursquare API was used to determine the most popular venues in the neighborhoods. These were then listed and sorted determining the most popular establishments per neighborhood. Once this was done a further evaluation of the neighborhoods which had in their top ten venues associated with Hispanic/Latin American culture were evaluated to determine potential rental listings. This was done by clustering the neighborhoods and determining which clusters are potentially more relevant to our search. An assumption was made to establish a cut off and limit the rental search using the most common venue as an indicator of community makeup. With this in mind a data-frame with the neighborhoods with Mexican restaurant as their most common venue was established.

## 4. Results

In this analysis we have explored the locations of Latin or Hispanic centric communities by assuming the most common venue establishes the majority of the make up of the area. Using clustering based on venues within walking distance (defined as 400m radius) we have clustered the neighborhoods into five groups. Of these groups, one has more than one instance of Hispanic communities staples like restaurants and and other amenities that imply the make-up of the community.

The largest clusters share a number of common features. Mexican restaurant, South American venues, as well as bakeries food trucks and grocery stores.

Being specific about the type of each area beyond these shared features is difficult but we note a few differences using the top ten venue types for each cluster:
These clusters showed a varied make up, the one which had most in common was cluster 8 where the majority of the determined neighborhoods of interest formed part of.

## 5. Discussion

Based on our results we have identified five neighborhoods where as per our assumption appeared to be Hispanic or Latin centric communities.

In theory, using these results, Mexican Restaurant could characterize other areas where Latin American or Mexican immigrants might consider for a potential area to rent or enjoy a night out eating and enjoying the community.

This data exploration is not significantly strong secondary correlations with other data sets should be employed to establish a strong correlation between the assumptions and the findings of this exercise.

In [124]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!pip install geopy

from geopy.geocoders import Nominatim

!pip install bs4

from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium
!pip install geocoder
print('Libraries imported.')

Libraries imported.


In [125]:
Kansas_City_df = pd.read_csv (r'KCMO_neighborhoods.csv')
Kansas_City_df.rename(columns={'GROUPTITLE': 'Neighborhood'}, inplace=True)

KC_df=Kansas_City_df.drop(columns=['ORGANIZATIONID', 'DATECHANGE','STATUS','GROUPID','GROUPWEBSITE','GROUPDESC', 'BOUNDN','BOUNDS','BOUNDW','BOUNDE','SHAPE.AREA','SHAPE.LEN','GROUPTYPE'])

KC_df.head()

,Neighborhood,Longitude,Latitude
0,Brittany Oaks Homes Association,-94.678455,39.243321
1,Foxcroft Homes Association,-94.604972,38.924482
2,Lea Manor Homes Association,-94.604405,38.950180
3,Glen Arbor Estates Homes Association,-94.596021,38.921821
4,Red Bridge Hills Homes Association,-94.585355,38.934943


In [126]:
address = 'Kansas City, MO'

geolocator = Nominatim(user_agent="kc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kansas City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kansas City are 39.100105, -94.5781416.


In [127]:
# create map of Kansas using latitude and longitude values
map_KC = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(KC_df['Latitude'], KC_df['Longitude'], KC_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_KC)  
    
map_KC

In [128]:
#define foursquare credentials for determining popular venues around each 
#neighborhood
CLIENT_ID = 'Removed' # your Foursquare ID
CLIENT_SECRET = 'Removed' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XSN5L32WC1XQZF3O0VJF1HR2MZWDC1BM4CSSGQIKTBTBOT51
CLIENT_SECRET:KUFYEBICKHCE25ORSEB3FAJMWWWT5FUXHIZBXX4X14E51DQI


In [129]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [130]:
KC_venues = getNearbyVenues(names=KC_df['Neighborhood'],
                                   latitudes=KC_df['Latitude'],
                                   longitudes=KC_df['Longitude']
                                  )

Brittany Oaks Homes Association
Foxcroft Homes Association
Lea Manor Homes Association
Glen Arbor Estates Homes Association
Red Bridge Hills Homes Association
Red Bud Estates Neighborhood Association
Plaza Westport Neighborhood Association
South Plaza Neighborhood Association
Sacred Heart Homes Association (Westside)
Westside Neighborhood Association
Valentine Neighborhood Association
Westside Planning Committee
Green Meadows Homes Association
Hickory Hollow Homes Association
Coventry-Baughman Neighborhood Association
Platte Brooke Homes Association
Barry Harbor/Hunter's Ridge Homes Association
Loma Vista West Townhouses
Original Loma Vista Neighborhood Association
Rockhill Ridge Neighborhood Association
Rockhill Crest Neighborhood Association
Manheim Park Neighborhood Association
Neighbors United for Action
Blue Hills Neighborhood Association
East Swope Highlands Neighborhood Association
Beacon Hill - McFeders Community Council (formerly Beacon Hill - McFeders Neighborhood Association

KeyError: 'groups'

In [131]:
# one hot encoding
KC_onehot = pd.get_dummies(KC_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
KC_onehot['Neighborhood'] = KC_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [KC_onehot.columns[-1]] + list(KC_onehot.columns[:-1])
KC_onehot = KC_onehot[fixed_columns]

KC_onehot.head()

,Neighborhood,ATM,Accessories Store,American Restaurant,Animal Shelter,Antique Shop,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Dog Run,Donut Shop,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hardware Store,Health & Beauty Service,Heliport,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotel Pool,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Library,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop,Motel,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pop-Up Shop,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Resort,Restaurant,River,Road,Rock Club,Roof Deck,Rugby Pitch,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Trail,Travel & Transport,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Vineyard,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Brittany Oaks Homes Association,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [132]:
KC_grouped = KC_onehot.groupby('Neighborhood').mean().reset_index()
KC_grouped

num_top_venues = 5

for hood in KC_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = KC_grouped[KC_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----40-46 North Neighborhood & Crime Watch----
                   venue  freq
0                   Pool  0.67
1   Gym / Fitness Center  0.33
2                    ATM  0.00
3                 Office  0.00
4  Outdoors & Recreation  0.00


----41/42 Montgall Homeowners Association----
          venue  freq
0        Garden   0.5
1  Liquor Store   0.5
2     Racetrack   0.0
3          Park   0.0
4        Museum   0.0


----433 Ward Parkway Condominium Association----
                 venue  freq
0  American Restaurant  0.10
1       Clothing Store  0.09
2        Women's Store  0.06
3   Seafood Restaurant  0.04
4                Hotel  0.04


----49/63 Coalition----
                      venue  freq
0            Discount Store   0.2
1                Restaurant   0.2
2  College Baseball Diamond   0.2
3                       Bar   0.2
4   Health & Beauty Service   0.2


----Amber Meadows/Amber Lakes Subdivisions----
                   venue  freq
0             Playground   0.5
1                    

In [133]:
#sort the data to determine the most common venues per neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = KC_grouped['Neighborhood']

for ind in np.arange(KC_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(KC_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,40-46 North Neighborhood & Crime Watch,Pool,Gym / Fitness Center,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Yoga Studio
1,41/42 Montgall Homeowners Association,Garden,Liquor Store,Yoga Studio,Fast Food Restaurant,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court
2,433 Ward Parkway Condominium Association,American Restaurant,Clothing Store,Women's Store,New American Restaurant,Hotel,Cosmetics Shop,Seafood Restaurant,Accessories Store,Coffee Shop,Sporting Goods Shop
3,49/63 Coalition,Discount Store,Health & Beauty Service,Restaurant,College Baseball Diamond,Bar,Yoga Studio,Fondue Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market
4,Amber Meadows/Amber Lakes Subdivisions,Playground,Gym,Food & Drink Shop,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Yoga Studio,Farmers Market


In [134]:
# set number of clusters
kclusters = 10

KC_grouped_clustering = KC_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(KC_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

KC_merged = KC_df

# merge KC_grouped with KC_df to add latitude/longitude for each neighborhood
KC_merged = KC_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

KC_merged.head() # check the last columns!


,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brittany Oaks Homes Association,-94.678455,39.243321,8.0,Rugby Pitch,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Farm
1,Foxcroft Homes Association,-94.604972,38.924482,8.0,BBQ Joint,Lawyer,Yoga Studio,Farmers Market,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop
2,Lea Manor Homes Association,-94.604405,38.950180,5.0,Pool,Construction & Landscaping,Trail,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant
3,Glen Arbor Estates Homes Association,-94.596021,38.921821,9.0,Bar,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court
4,Red Bridge Hills Homes Association,-94.585355,38.934943,8.0,Hotel,Breakfast Spot,Coffee Shop,Yoga Studio,Food & Drink Shop,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food


In [135]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(KC_merged['Latitude'], KC_merged['Longitude'], KC_merged['Neighborhood'], KC_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[-1 ],
        fill=True,
        fill_color=rainbow[-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [136]:
KC_merged.loc[KC_merged['Cluster Labels'] == 0, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Boston Heights and Mount Hope Neighborhood Ass...,Park,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food Court
42,Armour Hills Homes Association,Trail,Playground,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
58,Sunset Hill Neighborhood Association,American Restaurant,Athletics & Sports,Food & Drink Shop,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food Court
68,Waldo Homes Association,Playground,Comedy Club,Park,Food Service,Food Court,Food & Drink Shop,Food Truck,Food,Factory
84,Town Fork Creek Neighborhood Association,Baseball Field,Road,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Farm,Food
88,Forgotten Homes Neighborhood Association,Southern / Soul Food Restaurant,Park,BBQ Joint,Fondue Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Yoga Studio
89,Pendleton Heights Neighborhood Association,Disc Golf,Thrift / Vintage Store,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop
114,Scarritt Renaissance Neighborhood Association,Gym,Museum,Convenience Store,Taco Place,Electronics Store,Fondue Restaurant,Fast Food Restaurant,Fish & Chips Shop,Flea Market
115,Blue Valley Neighborhood Association,Disc Golf,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
117,Little Village Homes Association,Garden,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop


In [137]:
KC_merged.loc[KC_merged['Cluster Labels'] == 1, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Loma Vista West Townhouses,Pizza Place,Fried Chicken Joint,Sandwich Place,Grocery Store,Asian Restaurant,Discount Store,American Restaurant,Chinese Restaurant,Football Stadium
30,Green Haven Neighborhood Association,Pizza Place,Automotive Shop,Women's Store,American Restaurant,Discount Store,Department Store,Comic Shop,Pet Store,Fish & Chips Shop
33,Wedgewood Pointe Neighborhood Association,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
47,Walnut Grove Neighborhood Association,Garden Center,Yoga Studio,Food & Drink Shop,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food Service
49,Southern Hills Subdivision & Annex Homes Assoc...,Grocery Store,Coffee Shop,Yoga Studio,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Farmers Market
50,Birchwood Hills Homes Association,Performing Arts Venue,Yoga Studio,Food & Drink Shop,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food Court
57,Swope Parkway/Elmwood Neighborhood Association...,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
77,Tiffany Manor Neighborhood Association,Automotive Shop,Motel,Fried Chicken Joint,Gas Station,Steakhouse,Lounge,Sandwich Place,Lighting Store,Gym / Fitness Center
85,Oak Park Neighborhood Association,Fried Chicken Joint,Ice Cream Shop,Discount Store,Home Service,Yoga Studio,Food,Fish & Chips Shop,Flea Market,Flower Shop
86,Sheraton Estates Neighborhood Association,Sandwich Place,Discount Store,Shoe Store,Electronics Store,Shopping Plaza,Grocery Store,Cosmetics Shop,Women's Store,Wings Joint


In [138]:
KC_merged.loc[KC_merged['Cluster Labels'] == 2, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
128,Santa Fe Hills Homes Association,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Farm
197,Robincrest Village Homes Association,Trail,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Yoga Studio,Farmers Market
214,Shoal Brook II Neighborhood Association,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Farm


In [139]:
KC_merged.loc[KC_merged['Cluster Labels'] == 3, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,East 23rd St. PAC Neighborhood Association,Basketball Court,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service
112,Candlelight Ridge Homeowners Association,Food,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service
143,Gregory Ridge Neighborhood Association,Yoga Studio,Frozen Yogurt Shop,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
200,Lakeview Gardens Neighborhood Association,Yoga Studio,Frozen Yogurt Shop,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop


In [140]:
KC_merged.loc[KC_merged['Cluster Labels'] == 4, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
46,The Coves Homes Association,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
59,40-46 North Neighborhood & Crime Watch,Gym / Fitness Center,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Yoga Studio
67,Bridlespur Homes Association,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
137,Kirktown Homes Association,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
190,Summit Wood Property Owners Association,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
241,Willow Brooke Homes Association,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop


In [141]:
KC_merged.loc[KC_merged['Cluster Labels'] == 5, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Lea Manor Homes Association,Construction & Landscaping,Trail,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant
14,Coventry-Baughman Neighborhood Association,Gym,Park,Convenience Store,Construction & Landscaping,Fondue Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
15,Platte Brooke Homes Association,Pool,Food,Yoga Studio,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court
48,Marlborough East Neighborhood Association,Convenience Store,Construction & Landscaping,Food Court,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Yoga Studio
52,Sugar Tree Estates Homes Association,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
61,Carriage Hill Estates Homes Association,Pool,Yoga Studio,Food & Drink Shop,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food Court
110,Foxtown West Neighborhood Association,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
126,Woodson Estates Homes Association,Pharmacy,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
132,Janssen Place Lot Owners Association,Pool,Coffee Shop,Yoga Studio,Food,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop
138,Kirkside Homes Association,Home Service,Yoga Studio,Food & Drink Shop,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food Court


In [142]:
KC_merged.loc[KC_merged['Cluster Labels'] == 6, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
206,Seven Oaks Neighborhood Association (Knoches P...,Yoga Studio,Farmers Market,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
223,Country Lane Neighborhood Association,Assisted Living,Yoga Studio,Food,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Farmers Market


In [143]:
KC_merged.loc[KC_merged['Cluster Labels'] == 7, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Blue Hills Neighborhood Association,Frozen Yogurt Shop,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
123,Highland Acres-Lakeside Heights Homes Association,Frozen Yogurt Shop,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
207,South Round Top Neighborhood Association,Frozen Yogurt Shop,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food
209,Riverview Neighborhood Association,Frozen Yogurt Shop,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop,Food


In [144]:
KC_merged.loc[KC_merged['Cluster Labels'] == 8, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brittany Oaks Homes Association,Yoga Studio,Food,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Farm
1,Foxcroft Homes Association,Lawyer,Yoga Studio,Farmers Market,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop
4,Red Bridge Hills Homes Association,Breakfast Spot,Coffee Shop,Yoga Studio,Food & Drink Shop,Fish & Chips Shop,Flea Market,Flower Shop,Fondue Restaurant,Food
5,Red Bud Estates Neighborhood Association,Lighthouse,Restaurant,Food,Miscellaneous Shop,Sporting Goods Shop,Hotel,Fountain,Food Truck,Food Service
6,Plaza Westport Neighborhood Association,Arts & Crafts Store,Spa,Ice Cream Shop,Salon / Barbershop,New American Restaurant,Cocktail Bar,Business Service,Thai Restaurant,Massage Studio
7,South Plaza Neighborhood Association,Pizza Place,Sandwich Place,Yoga Studio,Electronics Store,Steakhouse,Breakfast Spot,Coffee Shop,Dive Bar,Park
8,Sacred Heart Homes Association (Westside),Coffee Shop,American Restaurant,Brewery,BBQ Joint,Office,Nightclub,Yoga Studio,Food & Drink Shop,Fondue Restaurant
9,Westside Neighborhood Association,South American Restaurant,Park,Art Gallery,Restaurant,Theater,Food Truck,Food Service,Football Stadium,Food Court
10,Valentine Neighborhood Association,Ethiopian Restaurant,Boutique,Sports Bar,New American Restaurant,Massage Studio,Martial Arts Dojo,Chinese Restaurant,Gay Bar,Fast Food Restaurant
11,Westside Planning Committee,Brewery,Taco Place,Restaurant,South American Restaurant,Dessert Shop,New American Restaurant,Office,Tapas Restaurant,Fountain


In [145]:
KC_merged.loc[KC_merged['Cluster Labels'] == 9, KC_merged.columns[[0] + list(range(5, KC_merged.shape[1]))]]

,Neighborhood,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Glen Arbor Estates Homes Association,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,Fountain,Football Stadium,Food Truck,Food Service,Food Court
164,Woodbridge Homes Association,Mexican Restaurant,Yoga Studio,Food Court,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Service


In [146]:
KC_ltnx= KC_merged.loc[(KC_merged['1st Most Common Venue']==('Mexican Restaurant'))]
KC_ltnx

,Neighborhood,Longitude,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Sacred Heart Homes Association (Westside),-94.600631,39.076984,8.0,Mexican Restaurant,Coffee Shop,American Restaurant,Brewery,BBQ Joint,Office,Nightclub,Yoga Studio,Food & Drink Shop,Fondue Restaurant
9,Westside Neighborhood Association,-94.600244,39.086082,8.0,Mexican Restaurant,South American Restaurant,Park,Art Gallery,Restaurant,Theater,Food Truck,Food Service,Football Stadium,Food Court
11,Westside Planning Committee,-94.597936,39.085088,8.0,Mexican Restaurant,Brewery,Taco Place,Restaurant,South American Restaurant,Dessert Shop,New American Restaurant,Office,Tapas Restaurant,Fountain
166,Wornall Pines Neighborhood Association,-94.594426,38.939253,8.0,Mexican Restaurant,Smoke Shop,Pizza Place,Trail,Nail Salon,Yoga Studio,Food,Fish & Chips Shop,Flea Market,Flower Shop
208,Indian Mound Neighborhood Association,-94.520484,39.111942,0.0,Mexican Restaurant,Park,Grocery Store,Farmers Market,Fountain,Football Stadium,Food Truck,Food Service,Food Court,Food & Drink Shop
